This notebooks is a set of different test for checking options for: 
1) Create / backtest strategies for BTC price charts
2) Extract statistics to see the probability of some scenarios

#### Imports

In [1]:
from datetime import timedelta
from typing import Tuple
from scipy.optimize import brute
from binance.client import Client
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import typing
import matplotlib as plt

coin="BTCUSDT"
api_key = "XySsyvxRCTGRFNUe900aspJDMAlfZhOYSQy3O70SlVoHM5yF6RpSSkpY4grvpS8R"
api_secret_key = "jDzM4LtdV4gQQxWRg3Uq59NMigOpFQsmMXNq6nN7ZLkcePe4vXLI2epcJMfqFTKU"
client = Client(api_key, api_secret_key)

#### Some helper functions and creation of Backtester class

In [2]:
def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 199
    display.width = 1000
    # display.precision = 2  # set as needed

set_pandas_display_options()

In [ ]:
class ToolboxPriceAnalitics(object):
    @staticmethod
    def dataframe_from_historicalKlines(data: list)-> pd.DataFrame:
        """Get data list from get_historical_Klines and creates a Dataframe.

        Args:
            data (list): A list of candles obtained using get_historical_Klines 

        Returns:
            pd.DataFrame: A dataframe containing these columns
            ["time", "open", "high", "low", "close", "volume", "close_time", "qav", "ntrades", "tbav", "tbqav", "ignore"]
        """
        data = pd.DataFrame(data, columns=["time", "open", "high", "low", "close", "volume", "close_time", "qav", "ntrades", "tbav", "tbqav", "ignore"])
        data = data.apply(pd.to_numeric)
        # Cleaning the columns that won't be needed
        data.drop("close_time", inplace=True, axis=1)
        data.drop("qav", inplace=True, axis=1)
        data.drop("tbav", inplace=True, axis=1)
        data.drop("tbqav", inplace=True, axis=1)
        data.drop("ignore", inplace=True, axis=1)
        return data
       
    @staticmethod
    def study_max_delta(data: pd.DataFrame, windows: int)->pd.DataFrame:
        """Includes in the dataframe (data) 4 columns per windows, calculating: 
        1) close_w_<#window>               :  close price in # window ahead
        2) close_delta_w_<#window>         :  delta between the current price and the same we will find in <#window> ahead
        3) close_delta_w_per_<#window>     :  delta in percentage
        4) close_low_consecutive_<#window> :  1 when all the previous candels delta where bellow the threshold

        Args:
            data (pd.DataFrame): A dataframe prepared with, at least, dataframe_from_historicalKlines columns
            windows (int): Number of windows that will be calculated

        Returns:
            pd.DataFrame: [description]
        """
        assert data.empty == False
        assert windows > 0
        for i in range(1, windows+1):
            data['close_next_cdl_'+str(i)]               = data['close'].shift(-i)
            data.fillna(0, inplace=True)
            data['close_delta_next_cdl_'+str(i)]         = data['close_next_cdl_'+str(i)]-data['close']
            data['close_delta_next_cdl_pert_'+str(i)]    = (data['close_next_cdl_'+str(i)]-data['close'])/data['close']*100
            #data['close_low_consecutive_'+str(i)] = np.where(data[])
        return data  
    
    @staticmethod
    def per(data: pd.DataFrame, windows, threshold):
        #data_1d_2y = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1DAY, "1095 day ago UTC")
        data = ToolboxPriceAnalitics.dataframe_from_historicalKlines(data)
        data = ToolboxPriceAnalitics.study_max_delta(data, 5, 10)
        data=data.loc[:,data.columns.str.startswith('close_delta_next_cdl_pert_')]
        data[data['close_delta_next_cdl_pert_1']>4][:-1]
    

In [59]:
class HVRJ_Backtester2(object):
    ''' Class for the vectorized backtesting of SMA-based trading strategies.
    '''
    def __init__(self, 
                 data: pd.DataFrame, 
                 price_column: str , 
                 window_size: int, 
                 start: datetime = None, 
                 end: datetime = None, 
                 lookback: int = 75):
        # initialize the object
        assert data is not None
        self.window_size=window_size
        self.data = data
        self.price_column = price_column
        self.start = start
        self.end = end
        self.results = None
        self.strategies = set()
        self.lookback = lookback
        
        # naive srategy is needed to be calculated only once
        self.set_strategy_parameters(1)
        
        
        if (self.window_size != 1):
            self.set_strategy_parameters(window_size)
        self.strategy_return = 0
        #assert self.start < self.end
        
    @staticmethod
    def prepare_data(data: list, lookback: int = 75)-> pd.DataFrame:
        """Will prepare a dataframe from to be used with the strategy Range High Volume 

        Args:
            data list: A list created from get_historical_klines 
            lookback (int, optional): Number of candles that will be use for the calculus of highVolume and lowVolume and the BO and BD. Defaults to 75.

        Returns:
            pd.DataFrame: a Dataframe containing all the data needed for being used with the class HRVJ_Backtesters 
        """
        lookback = lookback
    
        def column_from_series(s: pd.Series, column: str):
            hvb = data.loc[s.idxmax(), column]
            return hvb
        
        # Create the dataframe from the list containing the raw data
        data = ToolboxPriceAnalitics.dataframe_from_historicalKlines(data)
        
        # Calculate all the columns that will be needed
        data["highVB"] = data.volume.rolling(lookback).apply(column_from_series,  kwargs={'column':"high"}, raw=False)
        data["lowVB"] = data.volume.rolling(lookback).apply(column_from_series,  kwargs={'column':"low"}, raw=False)
        data["volMA"] = data.volume.rolling(20).mean()
        data["volNext"] = data.volume.shift(1)
        data["cloNext"] = data.close.shift(1)
        data["overMidBar"] = (data.close-data.low)/(data.high-data.low) > 0.5
        data["underMidBar"] = (data.close-data.low)/(data.high-data.low) < 0.5
        data["breakout"] = np.where((data.cloNext<data.highVB) & (data.close>data.highVB) & (data.volume>data.volMA) & (data.volume>data.volPrev) & (data.overMidBar),1,0)
        data["breakdown"] = np.where((data.cloNext>data.lowVB) & (data.close<data.lowVB) & (data.volume>data.volMA) & (data.volume>data.volPrev) & (data.underMidBar),1,0)
        
        data["time"] = data["time"].astype(str).apply(lambda x: datetime.fromtimestamp(int(x[:-3])))
        data.set_index("time", inplace=True)
        
        return data.fillna(0)
        
    def set_strategy_parameters(self, window_size: int) -> tuple:
        """Set all the parameters for the strategy: Volume Bar Breakout / Breakdown: 
        https://www.tradingview.com/script/V2cjsLhI-Volume-Bar-Breakout-and-Breakdown-Indicator/ 
        
        This class is an excuse for creating and testing a class where we are using a non-trivial strategy and how we can check the 
        strategy using a brute-force test.
        
        Note:
        The low volume of data allow me to include all the data in each row for easy testing, so instead of doing df.diff(), I'll store the
        columns and then substract it.

        Args:
            window_size (int): Size of the window (in candles) for testing te strategy.

        Returns:
            tuple: check run_strategy 
        """
        assert self.window_size > 0
        self.window_size = window_size
        
        if window_size not in self.strategies:
            self.strategies.add(window_size)
        else:
            return self.run_strategy()
        
        str_window_size = str(window_size)
        
        self.data['current_window'] = self.window_size
        self.data["cloFuture_"+str_window_size] = self.data.close.shift(-window_size)
        self.data["volFuture_"+str_window_size] = self.data.volume.shift(-window_size)

        # Storing the fail history
        self.data["fail_"+str_window_size] = 0
        self.data["fail_bo_"+str_window_size] = 0
        self.data["fail_bd_"+str_window_size] = 0
        self.data["fail_bo_"+str_window_size] = np.where((self.data.breakout==1) & (self.data.close > self.data["cloFuture_"+str_window_size]),
                                                           self.data["fail_"+str_window_size]+1,
                                                           self.data["fail_"+str_window_size])
        self.data["fail_bd_"+str_window_size] = np.where((self.data.breakdown==1) & (self.data.close < self.data["cloFuture_"+str_window_size]),
                                                           self.data["fail_"+str_window_size]+1,
                                                           self.data["fail_"+str_window_size])
        self.data["fail_"+str_window_size] = self.data["fail_bo_"+str_window_size]+self.data["fail_bd_"+str_window_size]
        
        # Storing the success
        self.data["succ_"+str_window_size] = 0
        self.data["succ_bo_"+str_window_size] = 0
        self.data["succ_bd_"+str_window_size] = 0
        self.data["succ_bo_"+str_window_size] = np.where((self.data.breakout) & (self.data.close < self.data["cloFuture_"+str_window_size]),
                                                          self.data["succ_"+str_window_size]+1,
                                                          self.data["succ_"+str_window_size])
        self.data["succ_bd_"+str_window_size] = np.where((self.data.breakdown) & (self.data.close > self.data["cloFuture_"+str_window_size]),
                                                          self.data["succ_"+str_window_size]+1,
                                                          self.data["succ_"+str_window_size])
        self.data["succ_"+str_window_size] = self.data["succ_bo_"+str_window_size]+self.data["succ_bd_"+str_window_size]
        
        # Storing the accumulate fail and success in price and percentage
        self.data["close_fail_bo_"+str_window_size] = np.where(self.data["fail_bo_"+str_window_size], 
                                                        self.data.breakout*(self.data["cloFuture_"+str_window_size]-self.data.close),
                                                        0)
        # We represent the fails with negative numbers, so when we have a close_succ_db even though the future_close-initial_close is positive, 
        # the loss needs to be presented with a negative (becasue we are in a sell)
        self.data["close_fail_bd_"+str_window_size] = np.where(self.data["fail_bd_"+str_window_size], 
                                                        -self.data.breakdown*(self.data["cloFuture_"+str_window_size]-self.data.close),
                                                        0)
        self.data["close_fail_"+str_window_size] =  self.data["close_fail_bo_"+str_window_size] + self.data["close_fail_bd_"+str_window_size]
    
        # this will be positive as needs to be presented in a successful breakout
        self.data["close_succ_bo_"+str_window_size] = np.where(self.data["succ_"+str_window_size],
                                                       self.data.breakout*(self.data["cloFuture_"+str_window_size]-self.data.close),
                                                       0)
        # this will be -1*X so positive as needs to be presented in a succcessful breakdown
        self.data["close_succ_bd_"+str_window_size] = np.where(self.data["succ_"+str_window_size],
                                                       -self.data.breakdown*(self.data["cloFuture_"+str_window_size]-self.data.close),
                                                       0)
        # this will be a positive number (or 0)
        self.data["close_succ_"+str_window_size] = self.data["close_succ_bo_"+str_window_size] + self.data["close_succ_bd_"+str_window_size]
        
        # The total strategy is to follow every single bo and bd
        self.data['total_return_strategy_'+str_window_size] = self.data['close_succ_'+str_window_size].cumsum() + self.data['close_fail_'+str_window_size].cumsum()
        
        # base case
        if (window_size==1):
            # Buy and Hold is begining - final price, always
            self.data['naivestrategy'] = self.data.iloc[-1]['close']-self.data.iloc[0]['close']
        
        # return for this windows size without strategy
        self.data['no_strategy_return_'+str_window_size] = (self.data[self.price_column].shift(-self.window_size) - self.data[self.price_column] ) / self.data[self.price_column]
        self.data.fillna(0, inplace=True)
        self.run_strategy()

    def run_strategy(self)-> tuple:
        ''' 
        Backtests the trading strategy.
        returns: (performance_strategy, performance_strategy - naive)
        '''
        str_ws = str(self.window_size)
        # gross performance of the strategy
        aperf = self.data['total_return_strategy_'+str_ws].iloc[-1]
        # out-/underperformance of strategy
        operf = aperf - self.data['naivestrategy'].iloc[-1]
        self.strategy_return = (round(aperf, 10), round(operf, 10))
        return self.strategy_return

    def plot_results(self):
        ''' Plots the cumulative performance of the trading strategy
        compared to the symbol.
        '''
        if self.results is None:
            print('No results to plot yet. Run a strategy.')
        title = '%s | SMA1=%d, SMA2=%d' % (self.symbol,
                                               self.SMA1, self.SMA2)
        self.results[['naivestrategy', 'cstrategy']].plot(title=title,
                                                     figsize=(10, 6))

    def update_and_run(self, parameter):
        ''' Updates parameters and returns negative absolute performance (for minimization algorithm).
        returns negative performance of the strategy 
        '''
        self.set_strategy_parameters(parameter[0])
        res = -self.run_strategy()[0]
        return res

    def optimize_parameters(self, parameter_range):
        ''' Finds global maximum given the SMA parameter ranges.
        Parameters: tuple
            tuples of the form (start, end, step size)
        '''
        opt = brute(self.update_and_run, (parameter_range,), finish=None)
        return opt, -self.update_and_run((int(opt),))
    
    def describe_strategy(self, window_size:int, force:bool=False)->dict:
        """ Set the window_size and run the strategy for returning the results of the strategy in a human-readable way

        Args:
            window_size (int): number of candles that will use in this strategy
            force (bool, optional): force the set of the window_size in case it is not

        Returns:
            dict: A dictionary containing all the strategy results for the current window_size
        """
        if window_size not in self.strategies and force:
            self.set_strategy_parameters(window_size)
        elif window_size not in self.strategies and not force:
            return {}
            
        res = {}
        str_ws         = str(window_size)
        num_fails_bo   = self.data["fail_bo_"+str_ws].sum()
        num_fails_bd   = self.data["fail_bd_"+str_ws].sum()
        num_fails      = self.data["fail_"+str_ws].sum()
        num_succ_bo    = self.data["succ_bo_"+str_ws].sum()
        num_succ_bd    = self.data["succ_bd_"+str_ws].sum()
        num_succ       = self.data["succ_"+str_ws].sum()
        total_signals  = num_fails+num_succ
        perct_fail             = 0 if total_signals == 0 else num_fails/total_signals
        perct_fail_bo_vs_total = 0 if total_signals == 0 else num_fails_bo / total_signals
        perct_fail_bo_vs_bo    = 0 if (num_fails_bo+num_succ_bo) == 0 else num_fails_bo / (num_fails_bo+num_succ_bo)
        perct_fail_bd_vs_total = 0 if total_signals == 0 else num_fails_bd / total_signals
        perct_fail_bd_vs_bd    = 0 if (num_fails_bd+num_succ_bd) == 0 else num_fails_bd / (num_fails_bd+num_succ_bd)
        perct_succ     = 0 if total_signals == 0 else 1-perct_fail
        price_fails    = self.data["close_fail_"+str_ws].sum()
        price_succ     = self.data["close_succ_"+str_ws].sum()
        price_fails_bo = self.data["close_fail_bo_"+str_ws].sum()
        price_fails_bd = self.data["close_fail_bd_"+str_ws].sum()
        price_succ_bo  = self.data["close_succ_bo_"+str_ws].sum()
        price_succ_bd  = self.data["close_succ_bd_"+str_ws].sum()
        res = { 'window_size'          : window_size,
                'total_signals'        : total_signals,
                'total_bo_signals'     : self.data['breakout'].sum(),
                'total_db_signals'     : self.data['breakdown'].sum(),
                'num_fails'            : num_fails, 
                'num_succ'             : num_succ,
                'num_fails_bo'         : num_fails_bo,
                'num_succ_bo'          : num_succ_bo,
                'perct_fail_bo'        : perct_fail_bo_vs_total,
                'price_fails_bo'       : price_fails_bo,
                'price_succ_bo'        : price_succ_bo,
                'perct_fail_bo_vs_bo'  : perct_fail_bo_vs_bo,
                'num_fails_bd'         : num_fails_bd,
                'num_succ_bd'          : num_succ_bd,
                'price_succ_bd'        : price_succ_bd,
                'price_fails_bd'       : price_fails_bd,
                'perct_fail_bd'        : perct_fail_bd_vs_total,
                'perct_fail_bd_vs_bd'  : perct_fail_bd_vs_bd,
                'total_perct_fail'     : perct_fail, 
                'total_perct_succ'     : perct_succ, 
                'total_price_fail'    : price_fails, 
                'total_price_succ'     : price_succ, 
                'total_max_pain_radio' : self.data['total_return_strategy_'+str_ws].min(),
                'total_max_gain_radio' : self.data['total_return_strategy_'+str_ws].max(),
                'total_return_strategy': self.data["total_return_strategy_"+str_ws].iloc[-1],
                'total_return_naive'   : self.data["naivestrategy"].iloc[-1]}
        return res
    
    def describe_strategies(self, windows_sizes: list, force:bool=False)->pd.DataFrame:
        res = [self.describe_strategy(i,force) for i in windows_sizes ] 
        return pd.DataFrame(res)

 Study of the strategy for 1Hour interval in the last 6 days. 1st we get the data, then we prepare the data and then we run the strategy

In [ ]:
data_1h_6d = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1HOUR, "6 day ago UTC")
data_n = HVRJ_Backtester2.prepare_data(data_1h_6d.copy())

In [60]:
strateg = HVRJ_Backtester2(data_n.copy(), 'close',1)
strateg.optimize_parameters((1,5,1))
strateg.describe_strategies(list(range(1,5)))

,window_size,total_signals,total_bo_signals,total_db_signals,num_fails,num_succ,num_fails_bo,num_succ_bo,perct_fail_bo,price_fails_bo,price_succ_bo,perct_fail_bo_vs_bo,num_fails_bd,num_succ_bd,price_succ_bd,price_fails_bd,perct_fail_bd,perct_fail_bd_vs_bd,total_perct_fail,total_perct_succ,total_price_fail,total_price_succ,total_max_pain_radio,total_max_gain_radio,total_return_strategy,total_return_naive
0,1,367,193,174,209,158,116,77,0.316076,-14323.56,13546.69,0.601036,93,81,13868.81,-12292.00,0.253406,0.534483,0.569482,0.430518,-26615.56,27415.50,-211.39,5736.02,799.94,-2347.57
1,2,367,193,174,204,163,113,80,0.307902,-22866.14,17598.36,0.585492,91,83,18867.43,-19227.08,0.247956,0.522989,0.555858,0.444142,-42093.22,36465.79,-5627.43,2803.30,-5627.43,-2347.57
2,3,367,193,174,204,163,108,85,0.294278,-23650.28,21716.25,0.559585,96,78,21941.76,-25228.07,0.261580,0.551724,0.555858,0.444142,-48878.35,43658.01,-5220.34,2626.00,-5220.34,-2347.57
3,4,367,193,174,189,178,106,87,0.288828,-27507.47,26359.97,0.549223,83,91,24810.68,-25727.49,0.226158,0.477011,0.514986,0.485014,-53234.96,51170.65,-2581.54,5541.42,-2064.31,-2347.57


strateg.describe_strategies(list(range(1,5))).plot(kind='bar',
                                                   x='window_size', 
                                                       y=['total_max_pain_radio', 'total_max_gain_radio', 'total_return_strategy', 'total_return_naive'],
                                                       title="cosas", 
                                                       figsize=(5,5))

# Study 1h interval

### In the last 365 days

In [28]:
data_1h_365d = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1HOUR, "365 day ago UTC")
data_1h_365d = HVRJ_Backtester2.prepare_data(data_1h_365d.copy())

In [73]:
strateg = HVRJ_Backtester2(data_1h_365d.copy(), 'close',1)
strateg.optimize_parameters((1,5,1))
strateg.describe_strategies(list(range(1,5)))

,window_size,total_signals,total_bo_signals,total_db_signals,num_fails,num_succ,num_fails_bo,num_succ_bo,perct_fail_bo,price_fails_bo,price_succ_bo,perct_fail_bo_vs_bo,num_fails_bd,num_succ_bd,price_succ_bd,price_fails_bd,perct_fail_bd,perct_fail_bd_vs_bd,total_perct_fail,total_perct_succ,total_price_fail,total_price_succ,total_max_pain_radio,total_max_gain_radio,total_return_strategy,total_return_naive
0,1,208,104,104,117,91,51,53,0.245192,-11162.07,17122.77,0.490385,66,38,14060.60,-19495.16,0.317308,0.634615,0.562500,0.437500,-30657.23,31183.37,-8558.54,4150.32,526.14,-2904.97
1,2,208,104,104,118,90,51,53,0.245192,-15970.83,23253.63,0.490385,67,37,16996.13,-26793.81,0.322115,0.644231,0.567308,0.432692,-42764.64,40249.76,-8443.53,3025.62,-2514.88,-2904.97
2,3,208,104,104,108,100,49,55,0.235577,-19224.89,25765.30,0.471154,59,45,16956.59,-31113.52,0.283654,0.567308,0.519231,0.480769,-50338.41,42721.89,-8721.14,3260.69,-7616.52,-2904.97
3,4,208,104,104,109,99,46,58,0.221154,-24284.34,27202.20,0.442308,63,41,21265.90,-40748.55,0.302885,0.605769,0.524038,0.475962,-65032.89,48468.10,-18613.53,291.18,-16564.79,-2904.97


### From Feb to Oct 2021 because it is a year with a low and a high in a few months

In [39]:
strateg = HVRJ_Backtester2(data_1h_365d.iloc[:-2640].copy(), 'close',1)
strateg.optimize_parameters((1,5,1))
strateg.describe_strategies(list(range(1,5)))

,window_size,total_signals,num_errors,num_succ,num_errors_bo,num_errors_bd,num_succ_bo,num_succ_bd,perct_error_bo,perct_error_bd,total_perct_error,total_perct_succ,total_price_error,total_price_succ,total_max_pain_radio,total_max_gain_radio,total_return_strategy,total_return_naive
0,1,138,77,61,34,43,40,21,0.246377,0.311594,0.557971,0.442029,-22196.28,18558.08,-8558.54,0.00,-3638.20,13563.26
1,2,138,77,61,33,44,41,20,0.239130,0.318841,0.557971,0.442029,-28960.55,25615.10,-8443.53,604.50,-3345.45,13563.26
2,3,138,70,68,32,38,42,26,0.231884,0.275362,0.507246,0.492754,-30685.00,28638.97,-6951.95,3260.69,-2046.03,13563.26
3,4,138,70,68,31,39,43,25,0.224638,0.282609,0.507246,0.492754,-44169.22,32660.27,-15822.31,291.18,-11508.95,13563.26


# Study 15 minutes

### Last 180 days

In [30]:
data_15m_180d = client.get_historical_klines(coin, Client.KLINE_INTERVAL_15MINUTE, "180 day ago UTC")

In [76]:
data_n = HVRJ_Backtester2.prepare_data(data_15m_180d.copy())
strateg = HVRJ_Backtester2(data_n.copy(), 'close',1)
strateg.optimize_parameters((1,5,1))
strateg.describe_strategies(list(range(1,5)))

,window_size,total_signals,total_bo_signals,total_db_signals,num_fails,num_succ,num_fails_bo,num_succ_bo,perct_fail_bo,price_fails_bo,price_succ_bo,perct_fail_bo_vs_bo,num_fails_bd,num_succ_bd,price_succ_bd,price_fails_bd,perct_fail_bd,perct_fail_bd_vs_bd,total_perct_fail,total_perct_succ,total_price_fail,total_price_succ,total_max_pain_radio,total_max_gain_radio,total_return_strategy,total_return_naive
0,1,367,193,174,209,158,116,77,0.316076,-14323.56,13546.69,0.601036,93,81,13868.81,-12292.00,0.253406,0.534483,0.569482,0.430518,-26615.56,27415.50,-211.39,5736.02,799.94,-2347.57
1,2,367,193,174,204,163,113,80,0.307902,-22866.14,17598.36,0.585492,91,83,18867.43,-19227.08,0.247956,0.522989,0.555858,0.444142,-42093.22,36465.79,-5627.43,2803.30,-5627.43,-2347.57
2,3,367,193,174,204,163,108,85,0.294278,-23650.28,21716.25,0.559585,96,78,21941.76,-25228.07,0.261580,0.551724,0.555858,0.444142,-48878.35,43658.01,-5220.34,2626.00,-5220.34,-2347.57
3,4,367,193,174,189,178,106,87,0.288828,-27507.47,26359.97,0.549223,83,91,24810.68,-25727.49,0.226158,0.477011,0.514986,0.485014,-53234.96,51170.65,-2581.54,5541.42,-2064.31,-2347.57


# Study 5 minutes interval

### Last 180 days

In [77]:
data_5m_180d = client.get_historical_klines(coin, Client.KLINE_INTERVAL_5MINUTE, "180 day ago UTC")

In [78]:
data_n = HVRJ_Backtester2.prepare_data(data_5m_180d.copy())
strateg = HVRJ_Backtester2(data_n.copy(), 'close',1)
strateg.optimize_parameters((1,5,1))
strateg.describe_strategies(list(range(1,5)))

,window_size,total_signals,total_bo_signals,total_db_signals,num_fails,num_succ,num_fails_bo,num_succ_bo,perct_fail_bo,price_fails_bo,price_succ_bo,perct_fail_bo_vs_bo,num_fails_bd,num_succ_bd,price_succ_bd,price_fails_bd,perct_fail_bd,perct_fail_bd_vs_bd,total_perct_fail,total_perct_succ,total_price_fail,total_price_succ,total_max_pain_radio,total_max_gain_radio,total_return_strategy,total_return_naive
0,1,1101,556,545,574,527,286,270,0.259764,-21330.31,25082.94,0.514388,288,257,22424.42,-23930.32,0.261580,0.528440,0.521344,0.478656,-45260.63,47507.36,-3796.75,2271.23,2246.73,-2071.13
1,2,1101,556,545,531,570,271,285,0.246140,-28061.48,35077.98,0.487410,260,285,36676.94,-30046.23,0.236149,0.477064,0.482289,0.517711,-58107.71,71754.92,-952.17,13782.89,13647.21,-2071.13
2,3,1101,556,545,572,529,288,268,0.261580,-35701.39,40513.33,0.517986,284,261,37796.70,-38193.54,0.257947,0.521101,0.519528,0.480472,-73894.93,78310.03,-2968.71,4433.85,4415.10,-2071.13
3,4,1101,556,545,581,520,284,272,0.257947,-40870.75,42840.63,0.510791,297,248,41025.03,-45727.33,0.269755,0.544954,0.527702,0.472298,-86598.08,83865.66,-6840.86,856.57,-2732.42,-2071.13


### Last 360 days

In [79]:
data_5m_360d = client.get_historical_klines(coin, Client.KLINE_INTERVAL_5MINUTE, "360 day ago UTC")

In [80]:
data_n = HVRJ_Backtester2.prepare_data(data_5m_360d.copy())
strateg = HVRJ_Backtester2(data_n.copy(), 'close',1)
strateg.optimize_parameters((1,5,1))
strateg.describe_strategies(list(range(1,5)))

,window_size,total_signals,total_bo_signals,total_db_signals,num_fails,num_succ,num_fails_bo,num_succ_bo,perct_fail_bo,price_fails_bo,price_succ_bo,perct_fail_bo_vs_bo,num_fails_bd,num_succ_bd,price_succ_bd,price_fails_bd,perct_fail_bd,perct_fail_bd_vs_bd,total_perct_fail,total_perct_succ,total_price_fail,total_price_succ,total_max_pain_radio,total_max_gain_radio,total_return_strategy,total_return_naive
0,1,2328,1138,1190,1283,1045,635,503,0.272766,-56775.06,50326.41,0.557996,648,542,61982.13,-69518.62,0.278351,0.544538,0.551117,0.448883,-126293.68,112308.54,-20028.62,790.00,-13985.14,-4392.48
1,2,2328,1138,1190,1225,1103,604,534,0.259450,-72846.49,71599.18,0.530756,621,569,85215.57,-94999.95,0.266753,0.521849,0.526203,0.473797,-167846.44,156814.75,-25631.07,1972.20,-11031.69,-4392.48
2,3,2328,1138,1190,1232,1096,607,531,0.260739,-86371.92,88066.90,0.533392,625,565,99977.98,-105695.69,0.268471,0.525210,0.529210,0.470790,-192067.61,188044.88,-11406.54,5240.22,-4022.73,-4392.48
3,4,2328,1138,1190,1270,1058,614,524,0.263746,-104030.71,94136.96,0.539543,656,534,105823.79,-128090.34,0.281787,0.551261,0.545533,0.454467,-232121.05,199960.75,-36268.74,1792.28,-32160.30,-4392.48


### Study about the max deltas between a candle and the previous one in the last 2 years

In [4]:
data_1h_2y = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1HOUR, "720 day ago UTC")
data_1h_2y = HVRJ_Backtester2.prepare_data(data_1h_2y)

In [6]:
data_1h_2y.head(1)

,open,high,low,close,volume,ntrades,highVB,lowVB,volMA,volPrev,cloPrev,overMidBar,underMidBar,breakout,breakdown
time,,,,,,,,,,,,,,,
2020-02-24 11:00:00,9717.81,9754.0,9698.04,9748.74,1241.652313,16300,0.0,0.0,0.0,0.0,0.0,True,False,0,0


In [100]:
temp = ToolboxPriceAnalitics.study_max_delta(data_1h_2y.copy(), 5, 10)

In [157]:
x=temp.loc[:,temp.columns.str.startswith('close_delta_next_cdl_pert_')]

In [113]:
x[:-5].max()

close_delta_next_cdl_pert_1    17.383989
close_delta_next_cdl_pert_2    25.095869
close_delta_next_cdl_pert_3    15.586931
close_delta_next_cdl_pert_4    19.993996
close_delta_next_cdl_pert_5    25.887998
dtype: float64

How many deltas there are with bigger than 5% of difference in the last 720 days?

In [158]:
x[abs(x['close_delta_next_cdl_pert_1'])>5][:-1]

,close_delta_next_cdl_pert_1,close_delta_next_cdl_pert_2,close_delta_next_cdl_pert_3,close_delta_next_cdl_pert_4,close_delta_next_cdl_pert_5
time,,,,,
2020-03-12 10:00:00,-18.211473,-17.502900,-18.031984,-18.495256,-17.133315
2020-03-12 23:00:00,-17.279887,-19.783549,-28.815207,-16.440450,-10.950764
2020-03-13 01:00:00,-11.259109,4.167598,11.011189,2.572872,6.483741
2020-03-13 02:00:00,17.383989,25.095869,15.586931,19.993996,25.887998
2020-03-13 03:00:00,6.569789,-1.530923,2.223478,7.244607,11.078411
2020-03-13 04:00:00,-7.601321,-4.078370,0.633218,4.230675,2.111757
2020-03-13 09:00:00,5.836388,6.044485,2.410365,4.713080,6.284991
2020-03-13 19:00:00,8.138663,7.444427,10.321262,12.354223,11.900091
2020-03-15 21:00:00,13.831094,3.496239,3.371683,1.317091,2.218675


Same but checking those where the delta was negative (red candle) and change is > 10%

In [135]:
x[x['close_delta_next_cdl_pert_1']<-10][:-1]

,close_delta_next_cdl_pert_1,close_delta_next_cdl_pert_2,close_delta_next_cdl_pert_3,close_delta_next_cdl_pert_4,close_delta_next_cdl_pert_5
time,,,,,
2020-03-12 10:00:00,-18.211473,-17.502900,-18.031984,-18.495256,-17.133315
2020-03-12 23:00:00,-17.279887,-19.783549,-28.815207,-16.440450,-10.950764
2020-03-13 01:00:00,-11.259109,4.167598,11.011189,2.572872,6.483741
